In [1]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from os import path

In [2]:
df = pd.read_csv("cumulative.csv")

In [3]:
df.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
drop_columns = ['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_pdisposition','koi_tce_delivname']

In [5]:
df = df.drop(columns = drop_columns)

In [6]:
df = df.dropna(axis = "columns", how = "all")

In [7]:
df = df.dropna()

In [8]:
df["koi_disposition"].value_counts()

FALSE POSITIVE    3763
CONFIRMED         2268
CANDIDATE         1772
Name: koi_disposition, dtype: int64

In [9]:
df.columns

Index(['koi_disposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_insol', 'koi_insol_err1',
       'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff',
       'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1',
       'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra',
       'dec', 'koi_kepmag'],
      dtype='object')

In [10]:
df.head()

,koi_disposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [11]:
# Assign X (data) and y (target)
X = df.drop("koi_disposition", axis=1)
y = df["koi_disposition"]
print(X.shape, y.shape)

(7803, 41) (7803,)


In [12]:
#Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## MinMax Scaler

In [13]:
#Fit on the training data
X_scaler = MinMaxScaler().fit(X_train)
X_scaler.get_params()

{'clip': False, 'copy': True, 'feature_range': (0, 1)}

In [14]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[0.        , 0.        , 1.        , ..., 0.87651957, 0.47862541,
        0.55029341],
       [0.01      , 1.        , 0.        , ..., 0.68650948, 0.53024592,
        0.43573849],
       [0.997     , 0.        , 0.        , ..., 0.45760972, 0.43391816,
        0.63104389],
       ...,
       [1.        , 0.        , 0.        , ..., 0.38216246, 0.63460776,
        0.68212249],
       [1.        , 0.        , 0.        , ..., 0.70011269, 0.46523085,
        0.56789817],
       [0.995     , 0.        , 0.        , ..., 0.29127159, 0.50939145,
        0.61492685]])

In [15]:
#Create a Logistic Regression Model
classifier = LogisticRegression()
classifier

LogisticRegression()

In [16]:
y_train.value_counts()

FALSE POSITIVE    2795
CONFIRMED         1704
CANDIDATE         1353
Name: koi_disposition, dtype: int64

In [17]:
y_test.value_counts()

FALSE POSITIVE    968
CONFIRMED         564
CANDIDATE         419
Name: koi_disposition, dtype: int64

In [18]:
#Fit (train) or model using the training data
classifier.fit(X_train_scaled,y_train)

C:\Users\yenia\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [19]:
#Validate the model using the test data
#classifier.score makes predictions
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.4613807245386193
Testing Data Score: 0.4797539723218862


In [20]:
# Create the SVC Model
# from sklearn.svm import SVC 
# model = SVC(kernel='linear')
# model

## Grid Search

In [21]:
yeni

NameError: name 'yeni' is not defined

In [22]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
param_grid = {'C': [1, 5, 10, 50],
              'penalty': ["l1", "l2"]}
model = LogisticRegression(solver = "liblinear")
grid = GridSearchCV(model, param_grid, verbose=3)

In [23]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ................................C=1, penalty=l1; total time=   1.1s
[CV 2/5] END ................................C=1, penalty=l1; total time=   1.4s
[CV 3/5] END ................................C=1, penalty=l1; total time=   1.5s
[CV 4/5] END ................................C=1, penalty=l1; total time=   1.3s
[CV 5/5] END ................................C=1, penalty=l1; total time=   1.1s
[CV 1/5] END ................................C=1, penalty=l2; total time=   0.1s
[CV 2/5] END ................................C=1, penalty=l2; total time=   0.1s
[CV 3/5] END ................................C=1, penalty=l2; total time=   0.1s
[CV 4/5] END ................................C=1, penalty=l2; total time=   0.1s
[CV 5/5] END ................................C=1, penalty=l2; total time=   0.1s
[CV 1/5] END ................................C=5, penalty=l1; total time=   8.5s
[CV 2/5] END ................................C=5,

GridSearchCV(estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [1, 5, 10, 50], 'penalty': ['l1', 'l2']},
             verbose=3)

In [24]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 50, 'penalty': 'l1'}


In [25]:
# List the best score
print(grid.best_score_)

0.8769632208573285


In [26]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [27]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

Test Acc: 0.368
